In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
# Load Data 
df = pd.read_csv('/kaggle/input/titanic/train.csv')
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
df.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [4]:
print('Age = 35  ==>', (df['Age']==35).sum())
print('Age = 22  ==>', (df['Age']==22).sum())
print('Age = 29  ==>', (df['Age']==29).sum())

Age = 35  ==> 18
Age = 22  ==> 27
Age = 29  ==> 20


So,
1.  ***some columns are not Important for train the model, like (names, Ticket, PassngerID)***
2.  ***we see also that there are some Missing data values in : ('age', 'Cabin', 'Embarked')***
3.  ***Dtype is not Clean (some are int64, some are float64, and some object)***

In [5]:
# we choose the only columns what we need for out Model
df = df[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked', 'Survived']]
df.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Survived
0,3,male,22.0,1,0,7.2500,S,0
1,1,female,38.0,1,0,71.2833,C,1
2,3,female,26.0,0,0,7.9250,S,1
3,1,female,35.0,1,0,53.1000,S,1
4,3,male,35.0,0,0,8.0500,S,0


In [6]:
# converting the Sex from male/female to 1/0 (int Dtype)

df['Sex'] = (df['Sex']=='male').astype(int)
df.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Survived
0,3,1,22.0,1,0,7.2500,S,0
1,1,0,38.0,1,0,71.2833,C,1
2,3,0,26.0,0,0,7.9250,S,1
3,1,0,35.0,1,0,53.1000,S,1
4,3,1,35.0,0,0,8.0500,S,0


In [7]:
# convert the Embarked from object to int using labelencoder

from sklearn.preprocessing import LabelEncoder

lbl_encdr = LabelEncoder()
df['Embarked']=lbl_encdr.fit_transform(df['Embarked'])


df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    891 non-null    int64  
 1   Sex       891 non-null    int64  
 2   Age       714 non-null    float64
 3   SibSp     891 non-null    int64  
 4   Parch     891 non-null    int64  
 5   Fare      891 non-null    float64
 6   Embarked  891 non-null    int64  
 7   Survived  891 non-null    int64  
dtypes: float64(2), int64(6)
memory usage: 55.8 KB


,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Survived
0,3,1,22.0,1,0,7.2500,2,0
1,1,0,38.0,1,0,71.2833,0,1
2,3,0,26.0,0,0,7.9250,2,1
3,1,0,35.0,1,0,53.1000,2,1
4,3,1,35.0,0,0,8.0500,2,0


In [8]:
# check for collumns with messing Values : 

df.isnull().sum()

Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Embarked      0
Survived      0
dtype: int64

we can see that there is 177 messing value in the Age column

In [9]:
# we fill the missing values Using the mean  
df['Age'].fillna(df['Age'].mean(), inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    891 non-null    int64  
 1   Sex       891 non-null    int64  
 2   Age       891 non-null    float64
 3   SibSp     891 non-null    int64  
 4   Parch     891 non-null    int64  
 5   Fare      891 non-null    float64
 6   Embarked  891 non-null    int64  
 7   Survived  891 non-null    int64  
dtypes: float64(2), int64(6)
memory usage: 55.8 KB


In [10]:
# convert all the values toy float
df = df.astype(float)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    891 non-null    float64
 1   Sex       891 non-null    float64
 2   Age       891 non-null    float64
 3   SibSp     891 non-null    float64
 4   Parch     891 non-null    float64
 5   Fare      891 non-null    float64
 6   Embarked  891 non-null    float64
 7   Survived  891 non-null    float64
dtypes: float64(8)
memory usage: 55.8 KB


***Now, We can see that there is no messing Values, and no Object Dtyape, and all are Float numbers***

In [11]:
# Split data to Train, and valid sets! 
np.random.seed(42)
Train, valid = np.split(df.sample(frac=1), [round(0.8*int(len(df)))])
np.random.seed()

In [12]:
print(Train.shape)
print(valid.shape)

(713, 8)
(178, 8)


In [13]:
# Defin the Features and Target in the sets, and Scale and oversample it ! 
# creat/defin the Function : 

from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import RandomOverSampler

def scal_ros_data(dataframe, oversample=True):
    X = dataframe[dataframe.columns[:-1]].values
    y = dataframe[dataframe.columns[-1]].values
    
    scaler = StandardScaler()
    X = scaler.fit_transform(X)
    
    if oversample : 
        ros = RandomOverSampler()
        X, y = ros.fit_resample(X, y)
        
    data = np.hstack((X, np.reshape(y, (-1,1))))
    
    return data, X, y 
    

In [14]:
# Call the Fonction : 
np.random.seed(42)
Train, X_Train, y_Train = scal_ros_data(Train, oversample=True)
Valid, X_Valid, y_Valid = scal_ros_data(valid, oversample=False)
np.random.seed()

In [15]:
print('Train shape :{} | X_train shape:{}, | y_Train shape:{}'.format(Train.shape, X_Train.shape, y_Train.shape))
print('Valid shape :{} | X_Valid shape:{}, | y_Valid shape:{}'.format(Valid.shape, X_Valid.shape, y_Valid.shape))

Train shape :(872, 8) | X_train shape:(872, 7), | y_Train shape:(872,)
Valid shape :(178, 8) | X_Valid shape:(178, 7), | y_Valid shape:(178,)


In [16]:
# Import Knn libraries 
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.model_selection import GridSearchCV


In [17]:
# defin the Values for the hyperparameters 

look_best_param = {
    'n_neighbors': [5, 10, 15, 35],
    'weights': ['uniform', 'distance'],
    'p': [1, 2],
}

# defin the Grid Searche CV 

grid_sear = GridSearchCV(KNeighborsClassifier(), look_best_param, cv=5)
grid_sear.fit(X_Train, y_Train)

# Defin the best Parameters 
best_param = grid_sear.best_params_ 

print(best_param)

{'n_neighbors': 10, 'p': 1, 'weights': 'distance'}


***The best VALUES ***

* **n_neighbors: 5**
* **p: 1**
* **weights: 'distance'**

In [18]:
# import KNN 

Model = KNeighborsClassifier(n_neighbors=80)
Model.fit(X_Train, y_Train)

KNeighborsClassifier(n_neighbors=80)

In [19]:
y_pred = Model.predict(X_Valid)

In [20]:
from sklearn.metrics import classification_report 

report = classification_report(y_pred, y_Valid)
print(report)

              precision    recall  f1-score   support

         0.0       0.81      0.87      0.83       105
         1.0       0.78      0.70      0.74        73

    accuracy                           0.80       178
   macro avg       0.79      0.78      0.79       178
weighted avg       0.80      0.80      0.80       178



In [21]:
# KNN MODEL-2 

Model_KNN_2 = KNeighborsClassifier(n_neighbors=50, weights ="distance", p=1,)
Model_KNN_2.fit(X_Train, y_Train)

y_pred_knn_2 = Model_KNN_2.predict(X_Valid)

report_knn_2 = classification_report(y_pred_knn_2, y_Valid)
print(report_knn_2)

              precision    recall  f1-score   support

         0.0       0.82      0.87      0.85       107
         1.0       0.78      0.72      0.75        71

    accuracy                           0.81       178
   macro avg       0.80      0.79      0.80       178
weighted avg       0.81      0.81      0.81       178



In [22]:
from sklearn.svm import SVC 

Model_svc = SVC(kernel='linear', C=0.8, class_weight='balanced')
Model_svc.fit(X_Train, y_Train)

SVC(C=0.8, class_weight='balanced', kernel='linear')

In [23]:
y_pred_svc = Model_svc.predict(X_Valid)

In [24]:
report_2 = classification_report(y_pred_svc, y_Valid)
print(report_2)

              precision    recall  f1-score   support

         0.0       0.87      0.84      0.85       117
         1.0       0.71      0.75      0.73        61

    accuracy                           0.81       178
   macro avg       0.79      0.80      0.79       178
weighted avg       0.81      0.81      0.81       178



***The best Accuracy is : 81% (Model : Model_SVC ) ***

In [25]:
# import the Test dataset 

df_test = pd.read_csv('/kaggle/input/titanic/test.csv')
df_test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [26]:
df_test_2 = df_test[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']]

# converting the Sex from male/female to 1/0 (int Dtype)

df_test_2['Sex'] = (df_test_2['Sex']=='male').astype(int)

# Encoder Embarked
lbl_encdr = LabelEncoder()
df_test_2['Embarked']=lbl_encdr.fit_transform(df_test_2['Embarked'])

#convert all to float
df_test_2 = df_test_2.astype(float)

# Fill the missing Values
df_test_2['Age'].fillna(df_test_2['Age'].mean(), inplace=True)
df_test_2['Fare'].fillna(df_test_2['Fare'].mean(), inplace=True)

df_test_2 = df_test_2.astype(float)


# Defind The Function of scalling data
def scalling(dataframe):
    X = dataframe[dataframe.columns].values
    
    scal = StandardScaler()
    X = scal.fit_transform(X)
    
    return X

# Call the finction 
X_Test = scalling(df_test_2)




/tmp/ipykernel_20/2145380589.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test_2['Sex'] = (df_test_2['Sex']=='male').astype(int)
/tmp/ipykernel_20/2145380589.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test_2['Embarked']=lbl_encdr.fit_transform(df_test_2['Embarked'])


In [27]:
#Predict Using SVC Model

test_pred = Model.predict(X_Test)
test_pred = test_pred.astype(int)

In [28]:
# Prepar the Output
output = pd.DataFrame({
    'PassengerId': df_test.PassengerId,
    'Survived' : test_pred
})

In [29]:
# Creat the CSV File for the prediction 
output.to_csv('submission.csv', index=False)

# Read the prediction Format
sub = pd.read_csv('submission.csv')

print('the data Shape :', sub.shape)
sub.head()

the data Shape : (418, 2)


,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1
